In [2]:
from google.colab import files
uploaded = files.upload()

Saving saved_model.zip to saved_model.zip


In [3]:
import shutil
import os

if not os.path.exists("saved_model"):
    os.makedirs("saved_model")

shutil.unpack_archive("saved_model.zip", "saved_model")

In [4]:
pip install flask flask-cors pyngrok librosa transformers torch numpy soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 773.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [5]:
from pyngrok import ngrok
ngrok.set_auth_token("your_auth_token")

In [ ]:
from flask import Flask, request, jsonify
import torch
import librosa
import numpy as np
import os
import json
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor
from flask_cors import CORS
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)

# Configure upload folder
UPLOAD_FOLDER = "uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Load your Kalmora model and processor
model_path = "./saved_model"  # Path to your saved model
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_path)
processor = Wav2Vec2Processor.from_pretrained(model_path)

# Load label map from saved model
try:
    with open(os.path.join(model_path, 'label_map.json'), 'r') as f:
        inverse_label_map = json.load(f)
    # Convert string keys to integers (JSON converts all keys to strings)
    inverse_label_map = {int(k): v for k, v in inverse_label_map.items()}
except FileNotFoundError:
    # Fallback to hardcoded map if file doesn't exist
    inverse_label_map = {
        0: 'ps',
        1: 'angry',
        2: 'happy',
        3: 'sad',
        4: 'fear',
        5: 'disgust',
        6: 'neutral'
    }

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def predict_emotion(file_path):
    try:
        # Load audio file
        speech, sr = librosa.load(file_path, sr=16000)

        # Pad/truncate to 32000 samples (2 seconds at 16kHz)
        max_length = 32000
        if len(speech) > max_length:
            speech = speech[:max_length]
        else:
            speech = np.pad(speech, (0, max_length - len(speech)), mode='constant')

        # Process audio
        inputs = processor(
            speech,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True
        ).to(device)

        # Predict
        with torch.no_grad():
            outputs = model(**inputs)
            predicted_class = outputs.logits.argmax().item()

        return {
            'emotion': inverse_label_map[predicted_class],
            'emotion_id': predicted_class
        }

    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return {"error": str(e)}

@app.route("/predict", methods=["POST"])
def predict():
    if "audio" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400

    file = request.files["audio"]
    if file.filename == "":
        return jsonify({"error": "Empty filename"}), 400

    try:
        # Save temporarily
        file_path = os.path.join(UPLOAD_FOLDER, file.filename)
        file.save(file_path)

        # Predict emotion
        result = predict_emotion(file_path)

        # Clean up
        os.remove(file_path)

        if "error" in result:
            return jsonify({"error": result["error"]}), 500

        return jsonify({
            "predictions": [{
                "filename": file.filename,
                "emotion": result["emotion"],
                "emotion_id": result["emotion_id"]
            }]
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/", methods=["GET"])
def home():
    return """
    <html>
        <head>
            <title>Kalmora Emotion Recognition API</title>
            <style>
                body { font-family: Arial, sans-serif; margin: 40px; line-height: 1.6; }
                h1 { color: #333; }
                pre { background: #f4f4f4; padding: 15px; border-radius: 5px; }
            </style>
        </head>
        <body>
            <h1>Kalmora Speech Emotion Recognition API</h1>
            <p>Upload an audio file to the /predict endpoint to get emotion predictions.</p>
            <h2>Example curl command:</h2>
            <pre>curl -X POST -F "audio=@your-audio-file.wav" {request.host_url}predict</pre>
            <h2>Available emotions:</h2>
            <ul>
                <li>ps (Pleasant Surprise)</li>
                <li>angry</li>
                <li>happy</li>
                <li>sad</li>
                <li>fear</li>
                <li>disgust</li>
                <li>neutral</li>
            </ul>
        </body>
    </html>
    """

if __name__ == "__main__":

    port = 5000
    public_url = ngrok.connect(port).public_url
    print(f" * Running on {public_url}")
    print(f" * Local URL: http://localhost:{port}")

    # Run Flask app
    app.run(port=port)

 * Running on https://5a35-35-229-172-59.ngrok-free.app
 * Local URL: http://localhost:5000
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
<ipython-input-6-a9754383de30>:49: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 07:50:57] "POST /predict HTTP/1.1" 200 -
<ipython-input-6-a9754383de30>:49: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.